In [ ]:
%load_ext autoreload
%autoreload 2

### IMPORTING FILES FROM THE PROJECT ROOT.

In [ ]:
import time
import numpy as np

#### --- MAKING ROOT PROJECT FOLDER VISIBLE AT SCRIPT LEVEL ---

##
## --- Finding root project folder ---
##     (needed to import settings)
##
import os, sys
def find_project_root(beacon="ROOT_BEACON"):
    current_path = os.getcwd()
    
    while True:
        if os.path.isfile(os.path.join(current_path, beacon)):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        if parent_path == current_path:
            raise RuntimeError(f"Unable to find project root.\nNo '{beacon}' file found in any parent directory.")
        
        current_path = parent_path

# --- Find and set project root ---
project_root = find_project_root()

if project_root not in sys.path:
    sys.path.insert(0, project_root)
####                    --- end ---


#### IMPORTING FROM ROOT PROJECT FOLDER 
from Utils import ArrayStorageCompressor, ArrayDataManager, ArraySettings
from Utils.Multiprocessing import MultiProcessing
from Utils import ArrayGenerator

### CALCULATE A, B COEFFICIENTS FOR THE FORMULA   $x_i = a * b^i$

In [ ]:
a = ArraySettings.ARRAY_START_KEY
b = np.exp((np.log(ArraySettings.ARRAY_END_KEY)-np.log(ArraySettings.ARRAY_START_KEY))/(ArraySettings.NUMBER_OF_SAMPLES-1))

print(a, b)

### CALCULATE KEYS BASED THE GEOMETRIC SERIES

In [ ]:
x_i = np.array([int(np.floor(a*np.pow(b, i))) for i in range(ArraySettings.NUMBER_OF_SAMPLES)])
print(x_i)

### GENERATE AND UPLOAD DATA ON ARRAY MANAGER

In [ ]:

job_arguments = []
# "D:\\Uniud\\II-Anno_24-25\\Linguaggi_di_Programmazione\\Progetto\\Array\\ArrayStorage\\N_variability\\storage_2025-05-20_18-28-51\\generation_params.json
parameters_create_function = ArrayGenerator.initialize_creation_parameters_environment()
for i in range(ArraySettings.NUMBER_OF_SAMPLES):
    n, m, rep, dtype, seeds = parameters_create_function(i, x_i[i])
    
    
    
    job_arguments.append(
        MultiProcessing.create_job_arguments(
            n= n,
            m= m,
            rep= rep,
            dtype= dtype,
            seeds= seeds
        )
    )


In [ ]:
%%time

arrayManager = ArrayDataManager.ArraySampleContainer()
def callback(i, result):
    arrayManager.update({
        
        int(x_i[i]): 
            ArrayDataManager.ArraySample(
                sample= result,
                creation_arguments= ArrayDataManager.ArraySampleCreationArguments(
                        **job_arguments[i].get_kwargs()
                    ),
                variability = int(x_i[i])
            )
    })

multiProcc = MultiProcessing(project_root_path=project_root)
multiProcc.startUniformMultiProcessing(ArrayGenerator.sample, job_arguments, callback, processes_number= 5)

print(f"[Status] Generated {len(arrayManager.keys())} samples over {ArraySettings.NUMBER_OF_SAMPLES} keys.")
print(f"[Memory] Total array data size: {arrayManager.estimate_data_size_MB():.3f} MB")

### SAVING ARRAYS ON LOCAL FILE

In [ ]:
storage_folder_path = ArraySettings.CREATE_ARRAY_STORAGE_FOLDER(project_root)

job_arguments = [
    MultiProcessing.create_job_arguments(
        data= sample_container,
        path= os.path.join(storage_folder_path, ArraySettings.CREATE_COMPRESSED_ARRAY_FILE_NAME(index))
    )
    for index, sample_container in enumerate(arrayManager.subdivideArrayUniformly(ArraySettings.MAX_ARRAY_SAVE_FILES))
]

multiProcc.startUniformMultiProcessing(ArrayStorageCompressor.writeOnFile, job_arguments, full_throttle= True)
print(f"Arrays compressed into: {storage_folder_path}")

ArrayStorageCompressor.writeOnFile(arrayManager.get_creation_arguments(), os.path.join(storage_folder_path, ArrayGenerator.ARRAY_GENERATION_FILE), as_json= True)
print(f"Array generation file location: { os.path.join(storage_folder_path, ArrayGenerator.ARRAY_GENERATION_FILE)}")

#### Shows elements distribution on saved files.

In [ ]:
import matplotlib.pyplot as plt
[plt.bar(i, sum(x.kwargs["data"].keys())) for i, x in enumerate(job_arguments)]
plt.show()